# Install Dependencies

In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


# GPU Check
* If the first number is not close to 0 MiB : kill the kernel
* eg. 12369MiB / 15109MiB : this is a lot of memory already used up, one cannot run the notebook, with the little memory thats left

In [3]:
!nvidia-smi

Tue Sep 26 16:18:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   35C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Base Model to use
model_name = "NousResearch/llama-2-7b-chat-hf"

# Instruction dataset : or your own
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-peft-finetuned-guanaco-llama2-1k"

## QLoRA parameters

In [6]:
# LoRA attention dimension:  rank of the update matrices : smaler => faster & fewer params to train
lora_r = 64

# Alpha parameter for LoRA scaling: The weight matrix is scaled by lora_alpha/lora_r
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

### bitsandbytes parameters

In [7]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False


### TrainingArguments parameters

In [8]:
# Output dir for model predictions and checkpoints 
output_dir = "./results"

# Number of training epochs, shoudl be adjsuted for max convergence
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size BS
per_device_train_batch_size = 4
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients 
# multiple small batches before performing a weight update
# done when the BS that fits in the memory is small
gradient_accumulation_steps = 1
gradient_checkpointing = True 
# Enable gradient checkpointing : technique to reduce memory usage by clearing activations of certain layers 
# & recomputing them during a backward pass; trades extra computation time for reduced memory usage


max_grad_norm = 0.3 # Maximum gradient normal for gradient clipping

learning_rate = 2e-4 # Initial LR (AdamW optimizer)
weight_decay = 0.001 # Weight decay to apply to all layers except bias/LayerNorm weights

optim = "paged_adamw_32bit" # The optimizer to use
lr_scheduler_type = "constant" # LR schedule (EMpericaly constant a bit better than cosine: try variations though)


max_steps = -1 # Number of training steps (if set: overrides num_train_epochs)
warmup_ratio = 0.03 # rampup from 0 to LR vs LR to min

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
save_steps = 25 # checkpointing
logging_steps = 25


### SFT parameters

In [9]:
# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

## Load dataset 
any additional preprocessing steps go here..

In [10]:
dataset = load_dataset(dataset_name, split="train")

Generating train split: 100%|██████████| 1000/1000 [00:00<00:00, 19684.91 examples/s]


In [11]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

### Check GPU compatibility with bfloat16

In [12]:
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

### Load base model

In [13]:

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards: 100%|██████████| 2/2 [00:49<00:00, 24.74s/it]


### Load LLaMA tokenizer

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

### Load LoRA configuration

In [15]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

### Set training parameters

In [16]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    # report_to="tensorboard"
)

### Set supervised fine-tuning parameters

In [17]:

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Map: 100%|██████████| 1000/1000 [00:00<00:00, 1534.03 examples/s]


In [18]:
%%time
# Train model ~ 30 mins
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.347300
50,1.616500
75,1.209800
100,1.440500
125,1.177300
150,1.361300
175,1.171900
200,1.460600
225,1.154500
250,1.529300


CPU times: user 14min 8s, sys: 9min 33s, total: 23min 42s
Wall time: 24min 29s


TrainOutput(global_step=250, training_loss=1.3469001846313478, metrics={'train_runtime': 1469.474, 'train_samples_per_second': 0.681, 'train_steps_per_second': 0.17, 'total_flos': 8755214190673920.0, 'train_loss': 1.3469001846313478, 'epoch': 1.0})

### Save trained model

In [19]:
trainer.model.save_pretrained(new_model)